<a href="https://colab.research.google.com/github/gokulakrishnanbalaji/Spam-classification-using-RNN/blob/main/Spam_Detection_using_RNN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##SPAM DETECTION 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**RNN** 

In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences
from keras.utils.data_utils import pad_sequences
import tensorflow as tf
from keras.layers import Dense

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

In [ ]:
df = pd.read_csv('spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df.shape

(5572, 2)

In [ ]:
df['target'] = df['Category'].map( {'spam':1, 'ham':0 })

In [ ]:

df_train = df.sample(frac=.8, random_state=11)
df_test = df.drop(df_train.index)
print(df_train.shape, df_test.shape)

(4458, 3) (1114, 3)


In [ ]:
y_train = df_train['target'].values
y_test = df_test['target'].values
y_train.shape,y_test.shape

((4458,), (1114,))

In [ ]:
X_train = df_train['Message'].values
X_test = df_test['Message'].values

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
word_dict = tokenizer.index_word


In [ ]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
print(X_train_seq[:5])
print(df_train.iloc[0,:])
for el in X_train_seq[0]:
    print(word_dict[el], end=' ')

[[667, 2, 233, 93, 640, 40, 132, 9, 51, 1252, 3, 51, 571, 2, 3823, 1020, 3824, 2596, 26, 188, 1427, 2, 1253, 69, 50, 86, 2, 3825], [1, 2016, 121, 404, 20, 157, 6, 3826, 19, 484, 1, 169, 322, 376, 6, 668, 170, 2017], [405, 74, 164, 4, 264, 15, 11, 2018], [51, 146, 2597, 30, 5, 938, 3827, 1121, 22, 12, 641, 71, 3828, 12, 356, 71, 3829, 2, 2598, 274, 938, 606, 12, 36, 154, 2019, 118, 260, 65, 189, 642, 377], [31, 3, 104, 1, 27, 807, 45, 43, 8, 4, 118]]
Category                                                 spam
Message     Welcome to UK-mobile-date this msg is FREE giv...
target                                                      1
Name: 4460, dtype: object
welcome to uk mobile date this msg is free giving you free calling to 08719839835 future mgs billed at 150p daily to cancel send go stop to 89123 

In [ ]:
len(X_train_seq)

4458

In [ ]:
X_train_pad = pad_sequences(X_train_seq, maxlen=20, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=20, padding='post')

X_train_pad.shape

(4458, 20)

In [ ]:
laenge_pads = 20
anz_woerter = 7982

lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=anz_woerter+1, output_dim=20, input_length=laenge_pads))
lstm_model.add(LSTM(400))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 20)            159660    
                                                                 
 lstm (LSTM)                 (None, 400)               673600    
                                                                 
 dense (Dense)               (None, 1)                 401       
                                                                 
Total params: 833,661
Trainable params: 833,661
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train_pad, y_train, epochs=10, batch_size=64,validation_data=(X_test_pad, y_test))

Epoch 1/10
70/70 [==============================] - 24s 198ms/step - loss: 0.1877 - accuracy: 0.9345 - val_loss: 0.0877 - val_accuracy: 0.9695
Epoch 2/10
70/70 [==============================] - 5s 76ms/step - loss: 0.0357 - accuracy: 0.9913 - val_loss: 0.1281 - val_accuracy: 0.9713
Epoch 3/10
70/70 [==============================] - 4s 56ms/step - loss: 0.0162 - accuracy: 0.9969 - val_loss: 0.3373 - val_accuracy: 0.9614
Epoch 4/10
70/70 [==============================] - 4s 54ms/step - loss: 0.0104 - accuracy: 0.9980 - val_loss: 0.0751 - val_accuracy: 0.9740
Epoch 5/10
70/70 [==============================] - 2s 25ms/step - loss: 0.0074 - accuracy: 0.9987 - val_loss: 0.0942 - val_accuracy: 0.9731
Epoch 6/10
70/70 [==============================] - 2s 24ms/step - loss: 0.0033 - accuracy: 0.9993 - val_loss: 0.0811 - val_accuracy: 0.9785
Epoch 7/10
70/70 [==============================] - 2s 28ms/step - loss: 0.0058 - accuracy: 0.9982 - val_loss: 0.2123 - val_accuracy: 0.9731
Epoch 8/10


In [ ]:
sms_test = ['hi, completed ml assignment?']
sms_seq = tokenizer.texts_to_sequences(sms_test)
sms_pad = pad_sequences(sms_seq, maxlen=20, padding='post')
# Use the predict() method to obtain the model's output probabilities
predictions=(lstm_model.predict(sms_pad) > 0.5).astype("int32")

# Print the predicted class label
print("Predicted class label:", predictions)


1/1 [==============================] - 0s 359ms/step
Predicted class label: [[0]]


In [ ]:
sms_test = ['Congratulations, you’ve won 10000 rupees. click the link']
sms_seq = tokenizer.texts_to_sequences(sms_test)
sms_pad = pad_sequences(sms_seq, maxlen=20, padding='post')

# Use the predict() method to obtain the model's output probabilities
predictions=(lstm_model.predict(sms_pad) > 0.5).astype("int32")

# Print the predicted class label
print("Predicted class label:", predictions)

1/1 [==============================] - 0s 18ms/step
Predicted class label: [[1]]


In [ ]:
scores2 = lstm_model.evaluate(X_test_pad,y_test,verbose=1,batch_size=5)
print('Accurracy: {}'.format(scores2[1]))

223/223 [==============================] - 1s 3ms/step - loss: 0.2354 - accuracy: 0.9713
Accurracy: 0.9712746739387512
